<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ca347102c617a34262e9c7d92742ccdca238adee3d3dbd8e0b73907d38bff534
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-16 15:19:00
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.42 C
-------------------
Today PnL: nan (nan%)
Current PnL: -17.11 L (-11.51%)
CY Booked + Current PnL: -6.08 L (-4.09%)
-------------------
Total profit:  2.25 L
Total loss:  -19.36 L
-------------------
Total Booked + Current PnL: 20.90 L (17.17%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.76%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.63 L (58.15%)
Deployed:  1.22 C
Current:  1.42 C
CAGR/XIRR %: 9.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COALINDIA,395.95,10.45,NaN,L-LC,NaN,146502.0,-3958.0,0.0,NaN,-2.63,0.00,-2.63,167.0,-inf,1.05,0.00,XY25,ATH,MINING
63,SBIN,863.00,-11.34,67.0,M-LC,4.26,220427.0,18948.0,8266.0,0.85,9.40,3.75,13.51,60.0,2.29,1.58,23.19,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.53,58.0,H-LC,11.05,172843.0,16666.0,9420.0,0.51,10.67,5.45,16.71,22.0,1.77,1.24,29.97,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,106.19,62.0,M-SC,4.41,88831.0,-11946.0,12037.0,0.89,-11.85,13.55,0.09,245.0,-0.99,0.63,16.70,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.70,114740.0,14236.0,16408.0,-0.14,14.16,14.30,30.49,79.0,0.87,0.82,51.16,MH,ATH,METALS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.93,54.0,H-SC,17.33,200008.0,54.0,97184.0,0.12,0.03,48.59,48.63,115.0,0.00,1.43,11.58,XR,NTT,MISC
35,ICICIPRULI,790.00,-20.65,44.0,H-MC,2.22,136459.0,671.0,42084.0,-0.23,0.49,30.84,31.48,107.0,0.02,0.98,12.67,X40,ATH,INSURANCE
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.82,180884.0,1076.0,111334.0,-0.32,0.60,61.55,62.52,88.0,0.01,1.29,34.89,XR,NTT,BANKS
52,PGHH,17973.08,-30.68,53.0,H-MC,4.37,200550.0,-270.0,69049.0,0.91,-0.13,34.43,34.25,80.0,-0.00,1.43,5.06,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-17.93,34.0,H-LC,5.64,295362.0,-20480.0,133149.0,-1.84,-6.48,45.08,35.67,5.0,-0.15,2.11,6.47,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-13.45,45.0,H-LC,8.15,218522.0,-33246.0,86447.0,-0.78,-13.21,39.56,21.13,27.0,-0.38,1.56,17.54,X40,ATH,PAINTS
44,JIOFIN,387.00,-0.10,50.0,H-LC,13.39,271416.0,4051.0,61394.0,0.08,1.52,22.62,24.48,48.0,0.07,1.94,57.32,XY24,BTT,FINANCE
20,CIPLA,1795.00,-20.12,51.0,H-LC,5.82,213556.0,9056.0,32354.0,0.68,4.43,15.15,20.25,10.0,0.28,1.53,13.55,X40N,BTT,PHARMA
40,INFY,2275.00,-17.44,53.0,H-LC,8.79,322141.0,9253.0,162423.0,0.27,2.96,50.42,54.87,3.0,0.06,2.30,9.80,X40,BTT,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-2.60,63.0,H-LC,1.01,241898.0,-4253.0,83455.0,0.64,-1.73,34.50,32.18,16.0,-0.05,1.73,33.69,X200,ATH,IT
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202226.0,2088.0,18807.0,0.22,1.04,9.30,10.44,4.0,0.11,1.45,6.66,X40,NTT,FMCG
51,NESTLEIND,1377.00,-7.84,60.0,H-LC,3.67,283725.0,18299.0,38501.0,-0.42,6.89,13.57,21.40,11.0,0.48,2.03,14.00,XY25,NTT,FMCG
84,VBL,671.64,-17.93,34.0,H-LC,5.64,295362.0,-20480.0,133149.0,-1.84,-6.48,45.08,35.67,5.0,-0.15,2.11,6.47,X40N,ATH,FMCG
20,CIPLA,1795.00,-20.12,51.0,H-LC,5.82,213556.0,9056.0,32354.0,0.68,4.43,15.15,20.25,10.0,0.28,1.53,13.55,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,62.53,69.0,H-SC,8.57,140746.0,-42029.0,153103.0,4.86,-23.00,108.78,60.77,125.0,-0.27,1.01,26.06,XR,NTT,JEWELLERY
77,TRIDENT,48.00,-5.57,64.0,M-SC,5.36,73557.0,-18698.0,45002.0,2.69,-20.27,61.18,28.51,224.0,-0.42,0.53,25.87,XR,NTT,TEXTILES
8,ATULAUTO,844.00,3910.81,65.0,M-SC,6.45,125068.0,-22300.0,74115.0,2.34,-15.13,59.26,35.16,236.0,-0.30,0.89,27.74,XY24,NTT,AUTO
55,RAJOOENG,143.33,-34.33,43.0,H-SC,10.66,99390.0,-3110.0,43940.0,2.25,-3.03,44.21,39.83,114.0,-0.07,0.71,3.89,AR,ATH,MISC
10,BAJAJHFL,152.00,-9.82,61.0,H-MC,6.07,189994.0,-10512.0,59278.0,2.21,-5.24,31.20,24.33,90.0,-0.18,1.36,9.56,X40N,BTT,FINANCE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.00,35.95,44.0,M-SC,8.45,179568.0,-82650.0,152471.0,-4.48,-31.52,84.91,26.63,229.0,-0.54,1.28,11.65,XY24,NTT,MISC
84,VBL,671.64,-17.93,34.0,H-LC,5.64,295362.0,-20480.0,133149.0,-1.84,-6.48,45.08,35.67,5.0,-0.15,2.11,6.47,X40N,ATH,FMCG
65,SHALBY,327.00,1065.87,65.0,M-SC,2.56,158607.0,-22536.0,68011.0,-1.66,-12.44,42.88,25.10,235.0,-0.33,1.13,27.11,XY24,NTT,HEALTHCARE
79,UJJIVANSFB,60.00,96.03,56.0,H-SC,12.42,124146.0,-18333.0,37852.0,-1.58,-12.87,30.49,13.70,163.0,-0.48,0.89,46.53,OX40N,NTT,BANKS
14,BERGEPAINT,680.00,-19.01,47.0,H-MC,6.01,217485.0,-9854.0,57916.0,-1.33,-4.33,26.63,21.14,106.0,-0.17,1.55,22.59,XY24,NTT,PAINTS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,106.19,62.0,M-SC,4.41,88831.0,-11946.0,12037.0,0.89,-11.85,13.55,0.09,245.0,-0.99,0.63,16.70,OX40N,NTT,DURABLES
79,UJJIVANSFB,60.0,96.03,56.0,H-SC,12.42,124146.0,-18333.0,37852.0,-1.58,-12.87,30.49,13.70,163.0,-0.48,0.89,46.53,OX40N,NTT,BANKS
19,CERA,9475.0,-20.05,46.0,H-SC,2.16,145866.0,-30037.0,72058.0,0.36,-17.08,49.40,23.89,149.0,-0.42,1.04,25.97,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-19.85,53.0,H-MC,7.80,108452.0,-23383.0,65603.0,0.40,-17.74,60.49,32.03,98.0,-0.36,0.78,21.48,OX40N,NTT,IT
45,KANSAINER,340.0,-67.97,55.0,H-SC,1.79,220878.0,-48789.0,85126.0,-0.30,-18.09,38.54,13.47,138.0,-0.57,1.58,11.72,XY24,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-25.80,56.0,H-LC,12.80,295912.0,-50044.0,124904.0,1.16,-14.47,42.21,21.64,1.0,-0.40,2.12,4.83,X40,ATH,IT
40,INFY,2275.00,-17.44,53.0,H-LC,8.79,322141.0,9253.0,162423.0,0.27,2.96,50.42,54.87,3.0,0.06,2.30,9.80,X40,BTT,IT
42,ITC,452.00,-37.50,54.0,H-LC,2.86,202226.0,2088.0,18807.0,0.22,1.04,9.30,10.44,4.0,0.11,1.45,6.66,X40,NTT,FMCG
84,VBL,671.64,-17.93,34.0,H-LC,5.64,295362.0,-20480.0,133149.0,-1.84,-6.48,45.08,35.67,5.0,-0.15,2.11,6.47,X40N,ATH,FMCG
1,ABB,7934.00,-38.50,63.0,H-LC,11.13,256934.0,-4685.0,123894.0,0.26,-1.79,48.22,45.57,7.0,-0.04,1.84,9.33,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,66.0,M-MC,6.97,228940.0,3978.0,163669.0,0.12,1.77,71.49,74.53,192.0,0.02,1.64,34.59,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,11.47,59.0,H-MC,3.69,192470.0,27800.0,40168.0,1.88,16.88,20.87,41.28,89.0,0.69,1.38,26.21,X40N,ATH,AC
30,HAVELLS,2069.16,1.62,66.0,H-MC,1.83,251800.0,4069.0,70982.0,1.18,1.64,28.19,30.30,92.0,0.06,1.80,15.15,X40,ATH,ELECTRICAL
85,VOLTAS,1918.49,1.16,59.0,H-MC,2.68,212250.0,20508.0,75519.0,-0.06,10.70,35.58,50.08,99.0,0.27,1.52,18.22,XY25,ATH,AC
24,DABUR,735.00,-3.66,54.0,H-MC,6.54,209566.0,12538.0,77078.0,-0.66,6.36,36.78,45.49,102.0,0.16,1.50,20.23,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.70,114740.0,14236.0,16408.0,-0.14,14.16,14.30,30.49,79.0,0.87,0.82,51.16,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-39.53,58.0,M-SC,0.64,102112.0,10614.0,68139.0,1.33,11.60,66.73,86.07,223.0,0.16,0.73,48.29,XR,NTT,DURABLES
37,INDIAMART,4850.92,-50.72,52.0,H-SC,12.19,137636.0,14300.0,119468.0,-0.47,11.59,86.80,108.45,119.0,0.12,0.98,36.54,AR,ATH,MISC
87,WIPRO,420.00,-11.52,53.0,M-LC,5.69,155794.0,4849.0,104600.0,-0.26,3.21,67.14,72.51,53.0,0.05,1.11,9.43,XR,NTT,IT
12,BANKINDIA,190.00,-30.43,61.0,H-MC,5.82,180884.0,1076.0,111334.0,-0.32,0.60,61.55,62.52,88.0,0.01,1.29,34.89,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,41.92,87072.0,-26477.0,66532.0,2.09,-23.32,76.41,35.27,268.0,-0.40,0.62,113.11,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.17,48.0,H-SC,13.34,89122.0,-11841.0,106376.0,0.54,-11.73,119.36,93.63,148.0,-0.11,0.64,32.74,SR,ATH,CHEMICALS
49,MASFIN,398.61,-15.54,58.0,H-SC,12.92,95820.0,-2160.0,23763.0,-0.16,-2.20,24.80,22.05,152.0,-0.09,0.68,39.28,XR,ATH,FINANCE
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.70,114740.0,14236.0,16408.0,-0.14,14.16,14.30,30.49,79.0,0.87,0.82,51.16,MH,ATH,METALS
8,ATULAUTO,844.00,3910.81,65.0,M-SC,6.45,125068.0,-22300.0,74115.0,2.34,-15.13,59.26,35.16,236.0,-0.30,0.89,27.74,XY24,NTT,AUTO


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.00,3910.81,65.0,M-SC,6.45,125068.0,-22300.0,74115.0,2.34,-15.13,59.26,35.16,236.0,-0.30,0.89,27.74,XY24,NTT,AUTO
22,COFFEEDAY,80.00,-41.65,55.0,L-SC,41.92,87072.0,-26477.0,66532.0,2.09,-23.32,76.41,35.27,268.0,-0.40,0.62,113.11,XR,NTT,HOTELS
50,NATIONALUM,247.44,-42.83,70.0,H-MC,5.70,114740.0,14236.0,16408.0,-0.14,14.16,14.30,30.49,79.0,0.87,0.82,51.16,MH,ATH,METALS
61,SAMMAANCAP,326.00,-170.45,62.0,M-SC,2.63,81726.0,-20484.0,113877.0,0.29,-20.04,139.34,91.37,208.0,-0.18,0.58,32.67,XY25,NTT,FINANCE
54,RAJESHEXPO,518.00,1881.06,66.0,L-SC,4.44,54057.0,-83120.0,83210.0,1.48,-60.59,153.93,0.07,267.0,-1.00,0.39,34.44,OX40N,NTT,JEWELLERY


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.50
1,25,42.79
2,50,73.20


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.25
LC    34.03
MC    22.77
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.44
X40      15.08
X40N     12.79
XR       11.37
XY25      9.63
AR        8.54
OX40N     8.06
X200      1.73
MH        1.70
X5K       1.44
SR        1.27
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.43
H-LC    25.91
H-MC    19.86
M-SC    13.29
M-LC     7.07
M-MC     2.61
L-SC     1.53
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.20,-13.99,76.14
FMCG,11.61,-4.23,40.70
FINANCE,9.51,-15.61,60.89
BANKS,7.64,-12.27,63.95
MISC,7.30,-12.35,74.83
PAINTS,5.70,-15.78,33.04
ELECTRICAL,5.62,-4.85,62.63
AC,3.60,3.60,29.04
INSURANCE,3.38,-6.27,42.72


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2864221.0,22
XR,1318749.0,14
AR,1303507.0,9
X40,740708.0,10
X40N,625577.0,10
OX40N,520760.0,10
XY25,367064.0,7
SR,244323.0,2
X5K,116468.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3437786.0,29
M-SC,1384977.0,17
H-LC,1254445.0,15
H-MC,1175799.0,15
M-LC,389426.0,5
M-MC,312613.0,2
L-SC,247384.0,3
L-MC,60307.0,1
L-LC,0.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1035491.0      6
M-SC       XY24       805016.0      7
H-SC       AR         784603.0      5
           XR         765193.0      7
H-MC       XY24       489615.0      4
H-LC       X40        464784.0      5
           AR         356587.0      2
M-MC       XY24       312613.0      2
H-SC       X40N       297254.0      4
           OX40N      249425.0      4
           SR         244323.0      2
H-LC       X40N       228877.0      4
H-MC       X40        207820.0      4
           XY25       185573.0      2
L-SC       XR         164174.0      2
M-SC       AR         162317.0      2
M-LC       XY24       160092.0      2
M-SC       OX40N      122522.0      4
M-LC       X5K        116468.0      1
M-SC       XY25       113877.0      1
           XR         113141.0      2
H-MC       XR         111334.0      1
M-LC       XR         104600.0      1
H-MC       X40N        99446.0      2
H-LC       X200        83455.0      1
L-SC       OX40N       83210.0      1
M-SC       X40         68104.0      1
H-MC       OX40N       65603.0      1
H-SC       MH          61497.0      1
H-LC       XY24        61394.0      1
L-MC       XR          60307.0      1
H-LC       XY25        59348.0      2
H-MC       MH          16408.0      1
M-LC       XY25         8266.0      1
L-LC       XY25            0.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 64.0 seconds
